# Import raw observation data

Our raw data is from camera traps, each observation has a set of UTM coordinates as well as the species name.

In [26]:
import pandas as pd
import chardet

filepath = "../data/raw_cameratrap.csv"

# Detect the encoding of the file
with open(filepath, 'rb') as f:
    result = chardet.detect(f.read())

# Read the CSV file into a pandas DataFrame using the detected encoding
df = pd.read_csv(filepath, encoding=result['encoding'])

# Preview the first 5 rows of the DataFrame
print(df.head())

                         Survey Name    Common              Species  \
0  Elephants camera trapping project  Athérure  Atherurus africanus   
1  Elephants camera trapping project  Athérure  Atherurus africanus   
2  Elephants camera trapping project  Athérure  Atherurus africanus   
3  Elephants camera trapping project  Athérure  Atherurus africanus   
4  Elephants camera trapping project  Athérure  Atherurus africanus   

        Date      Time DayNight         X          Y  Elevation Habitat  ...  \
0  30-Aug-17  03:42:18    Night  507945.5  120331.10      366.0  Forest  ...   
1  23-Sep-17  23:37:24    Night  487829.4   21227.15        NaN  Forest  ...   
2  20-Dec-17  19:41:44    Night  487829.4   21227.15        NaN  Forest  ...   
3  23-Dec-17  20:41:40    Night  487829.4   21227.15        NaN  Forest  ...   
4  14-Jun-17  22:56:12    Night  533717.5   85685.04      534.8  Forest  ...   

   Group2      Sex Individuals  StationID   CamNumber1   CamNumber2  \
0     NaN      NaN   

/tmp/ipykernel_706/2275657478.py:11: DtypeWarning: Columns (18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, encoding=result['encoding'])


Next, we transform the UTM coordinates to WGS84 lat, lon.

In [ ]:
# Select columns of interest
col_list = ['Common', 'Species', 'Date', 'X', 'Y']
df_subset = df[col_list]

df_subset['Date'] = pd.to_datetime(df_subset['Date'])
df_subset['Year'] = df_subset['Date'].dt.year

# Transform UTM to lat, lon
import pyproj

# Define the UTM zone and hemisphere (N/S)
utm_zone = 33
utm_hemisphere = 'N'

# Create a pyproj Transformer object for the UTM projection
utm_crs = pyproj.CRS.from_proj4(f'+proj=utm +zone={utm_zone} +{utm_hemisphere} +ellps=WGS84 +datum=WGS84 +units=m +no_defs')

# Create a pyproj Transformer object for the WGS 84 Latitude-Longitude projection
lat_lon_crs = pyproj.CRS.from_proj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')

# Create a Transformer object to convert from UTM to Latitude-Longitude
transformer = pyproj.Transformer.from_crs(utm_crs, lat_lon_crs)

def convert_coordinates(row):
    lat, lon = transformer.transform(row['X'], row['Y'])
    return pd.Series({'Latitude': lat, 'Longitude': lon})

# Apply the function to all rows in the dataframe
df_subset[['Latitude', 'Longitude']] = df_subset.apply(convert_coordinates, axis=1)
print(df.head())

# Summarise the Species column
# value_counts = df_subset.groupby(['Common', 'Year']).size().reset_index(name='counts')
# value_counts = value_counts.pivot_table(values="counts", index="Common", columns="Year")

# print(value_counts)

In [25]:
import pandas as pd
import chardet

filepath = "../data/raw_dung.csv"

# Detect the encoding of the file
with open(filepath, 'rb') as f:
    result = chardet.detect(f.read())

# Read the CSV file into a pandas DataFrame using the detected encoding
df_dung = pd.read_csv(filepath, encoding=result['encoding'])

# Select columns of interest
col_list = ['Espèces', 'year', 'X', 'Y']
df_dung = df_dung[col_list]
df_dung = df_dung.rename(columns={'Espèces': 'Species', 'year': 'Year', 'X': 'Latitude', 'Y': 'Longitude'})



/tmp/ipykernel_706/3154978916.py:11: DtypeWarning: Columns (15,19,30,32,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dung = pd.read_csv(filepath, encoding=result['encoding'])


In [28]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd

# Convert to geopandas
ll_crs = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'
df_dung.crs = ll_crs

# Create a geometry column from the latitude and longitude values
df_dung['geometry'] = df_dung.apply(lambda x: Point(x.Latitude, x.Longitude), axis=1)

# Transform the pandas DataFrame to a GeoPandas DataFrame
gdf_dung = gpd.GeoDataFrame(df_dung, geometry='geometry')



# Clip points to area of interest

In [61]:
# Join cameratrap and gdung df

# Convert cameratrap to geopandas df
df_subset = df_subset[['Species', 'Year', 'Latitude', 'Longitude']]

df_subset['geometry'] = df_subset.apply(lambda x: Point(x.Latitude, x.Longitude), axis=1)

gdf = gpd.GeoDataFrame(df_subset, geometry='geometry')

combined_gdf = pd.concat([gdf, gdf_dung])

# Clean data
combined_gdf.loc[combined_gdf['Species']=="Loxodonta africana (éléphant Africain, éléphant d'Afrique) E", 'Species'] = 'Loxodonta cyclotis'
combined_gdf.loc[combined_gdf['Species']=="Loxodonta africana (Eléphant Africain, Eléphant d'Afrique) E", 'Species'] = 'Loxodonta cyclotis'
combined_gdf.loc[combined_gdf['Species']=="Loxodonta", 'Species'] = 'Loxodonta cyclotis'
combined_gdf.loc[combined_gdf['Species']=="Panthera pardus (Léopard, Panthère) PP", 'Species'] = 'Panthera pardus'
combined_gdf.loc[combined_gdf['Species']=="Pan troglodytes ssp. troglodytes (Chimpanzé) PT", 'Species'] = 'Pan troglodytes'
combined_gdf.loc[combined_gdf['Species']=="Gorilla gorilla ssp. gorilla (Gorille) GG", 'Species'] = 'Gorilla gorilla gorilla'

combined_gdf.crs = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'



In [71]:
import geopandas as gpd
from shapely import Point


# Create a function to to subset the data by species and year
def subset_species(df, species, year):

    # Filter by species
    try:
        df_species = df[(df['Species'] == species) & (df['Year'] == year)]

        # Import shapefile of park
        park = gpd.read_file('../data/odzala.geojson')
        park = park.to_crs(df_species.crs)

        # Clip points to park
        gdf = df_species[df_species.geometry.intersects(park.geometry.iloc[0])]
    
    except:
        gdf = gpd.GeoDataFrame()

    return gdf




               Species  Year   Latitude  Longitude                  geometry
81197  Panthera pardus  2021  15.069695   1.086870  POINT (15.06969 1.08687)
81235  Panthera pardus  2021  15.097350   1.149309  POINT (15.09735 1.14931)
81236  Panthera pardus  2021  15.097350   1.149309  POINT (15.09735 1.14931)
81237  Panthera pardus  2021  15.097350   1.149309  POINT (15.09735 1.14931)
81257  Panthera pardus  2021  15.175696   0.905955  POINT (15.17570 0.90595)


# Create a buffer around points
Here we create a 10km2 buffer around points

In [ ]:
def create_buffer(gdf, dist_km):

    # Store the current crs
    ll_crs = gdf.crs

    # Reproject to projected to Africa Sinusoidal projection for accurate distance calculations
    as_proj = '+proj=sinu +lon_0=15 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs'
    as_gdf = gdf.to_crs(as_proj)

    # Create a buffer around each point
    buffer_gdf = as_gdf
    buffer_gdf['geometry'] = as_gdf.buffer(dist_km*1000)

    # Reproject back to lat, lon
    ll_gdf = buffer_gdf.to_crs(ll_crs)

    return ll_gdf


# Import raster

In [ ]:
import rasterio
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
from rasterio import Affine

# Input raster and resample to 1000m (0.01)

def resample_raster(input_raster, res, output_name):

    xres, yres = res, res

    with rasterio.open(input_raster) as dataset:
        scale_factor_x = dataset.res[0]/xres
        scale_factor_y = dataset.res[1]/yres

        profile = dataset.profile.copy()
        # resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * scale_factor_y),
                int(dataset.width * scale_factor_x)
            ),
            resampling=Resampling.bilinear
        )

        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (1 / scale_factor_x),
            (1 / scale_factor_y)
        )
        profile.update({"height": data.shape[-2],
                        "width": data.shape[-1],
                    "transform": transform})

    with rasterio.open(output_name, "w", **profile) as dataset:
        dataset.write(data)


# resample_raster("../data/odzala/odzala_humanfootprint_18.tif", 0.01, "../data/odzala/output18.tif")



In [ ]:
# with rasterio.open('../data/odzala/output.tif') as src:
#     image = src.read(1)
#     plt.imshow(image, cmap='gray')
#     plt.colorbar()
#     plt.show()

# Transform polygons to raster

In [ ]:
from rasterio.features import geometry_mask

def poly_to_raster(gdf, input_raster, output_name):
    with rasterio.open(input_raster) as src:
        image = src.read(1)

    mask = geometry_mask(gdf["geometry"], transform=src.transform, out_shape=src.shape)
    image[~mask] = 1
    image[mask] = 0

    meta = src.meta.copy()
    meta.update(compress='lzw')

    # Save the raster to a TIFF file
    with rasterio.open(output_name, "w", **meta) as dst:
        dst.write(image, 1)


# poly_to_raster(test2, '../data/odzala/output18.tif', "../data/odzala/leopard.tif")

# Use the resample_raster function to upsample the human-footprint
Here, we read the 30m human-footprint rasters and then apply the `resample_raster` function to upsample each raster to 1000m

In [ ]:
# Resample all 30m human_footprint rasters to 1000m

# List all files
import os
directory = '../data/odzala/human_footprint_30/'

# Get a list of all files in the directory and their full paths
files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

for file_path in files:

    # Get just the filename from the file path and remove extension
    filename = os.path.basename(file_path)
    filename = os.path.splitext(filename)[0]
    
    # Resample raster and write to different folder
    resample_raster(file_path, 0.01, "../data/odzala/human_footprint_1000/" + filename + "_1000.tif")



# Create rasters from species polygons
Here, we apply the `subset_species`, `create_buffer` and `poly_to_raster` functions to process the observation data into rasters. Each raster is specific to a species and year.

In [ ]:
# Create rasters of species data
import os

key_species = ['Loxodonta cyclotis', 'Panthera pardus', 'Pan troglodytes', 'Gorilla gorilla gorilla', 'Hylochoerus meinertzhageni']
years = [2017, 2018, 2019, 2020, 2021]


for species in key_species:
    for year in years:
        df_species = subset_species(df_subset, species, year)

        if not df_species.empty:
            df_buffer = create_buffer(df_species, 10)

            output_folder_path = '../data/odzala/species/' + str(year)

            if not os.path.exists(output_folder_path):
                os.makedirs(output_folder_path)
            
            poly_to_raster(df_buffer, '../data/odzala/human_footprint_1000/odzala_humanfootprint_17_1000.tif', output_folder_path + '/' + str(year) + '_' + species + '.tif')



# Now combine the species rasters into one biodiversity raster

We want to know how many of the key species were observed in a given year. As such, we sum the values for each species raster and create an aggregate raster for the year.

In [ ]:
import os
import rasterio
import numpy as np

# List all rasters in the folder
years = [2017, 2018, 2019, 2020, 2021]


def sum_rasters(year):
    folder = '../data/odzala/species/' + str(year)
    rasters = [f for f in os.listdir(folder) if f.endswith(".tif")]

    # Read the first raster and get its shape and CRS
    with rasterio.open(os.path.join(folder, rasters[0])) as src:
        shape = src.shape
        crs = src.crs
        first_raster = src.read(1)

    # Initialize the sum with the first raster
    result = first_raster.astype(float)

    # Loop through the remaining rasters and add them to the sum
    for raster in rasters[1:]:
        with rasterio.open(os.path.join(folder, raster)) as src:
            result += src.read(1).astype(float)

    # Write the sum to a new raster
    filename = '../data/odzala/species/Summed/'+ str(year) + '.tif'
    with rasterio.open(filename, 'w', driver='GTiff', height=shape[0],
                    width=shape[1], count=1, dtype=np.float32,
                    crs=crs, transform=src.transform) as dst:
          dst.write(result.astype(np.float32), 1)


for year in years:
    sum_rasters(year)

# Create maps
First, we create maps of the human-footprint mapping.

In [ ]:
import matplotlib.pyplot as plt

folder = '../data/odzala/human_footprint_1000/'
rasters = sorted([f for f in os.listdir(folder) if f.endswith(".tif")])

# Read shapefile of park
park = gpd.read_file('../data/odzala.geojson')

fig, axes = plt.subplots(1, 5, figsize=(20, 5))
cmap = plt.get_cmap("Reds")

for i, ax in enumerate(axes):
    with rasterio.open(os.path.join(folder, rasters[i])) as src:
        title = '20' + rasters[i].split("_")[2]
        image = src.read(1)
        image = np.ma.masked_where(image < 0.002, image) # 0.002 indicates more than two 30m pixels to be deforested within the 1000m pixel.

        # Transform values to exponential if of interest
        # image = np.exp(image)

        extent = src.bounds

        park = park.to_crs(src.crs)

        ax.imshow(image, extent=(extent.left, extent.right, extent.bottom, extent.top), origin='upper', cmap='Reds')
        park.boundary.plot(ax=ax, color='green', linewidth=0.5)
        ax.set_title(title, fontsize=20)
        ax.set_axis_off()

# cbar = fig.colorbar(axes[4].imshow(image, cmap=cmap), ax=axes, orientation="horizontal", fraction=0.046, pad=1, anchor=(0.5, -0.1))
# cbar.ax.tick_params(labelsize=18)

plt.tight_layout()
# plt.show()
plt.savefig('../data/odzala/plots/humanfootprint_linear.png', bbox_inches='tight', dpi=300)



Then, we create maps for the species observations

In [ ]:
import matplotlib.pyplot as plt

folder = '../data/odzala/species/Summed/'
rasters = sorted([f for f in os.listdir(folder) if f.endswith(".tif")])

# Read shapefile of park
park = gpd.read_file('../data/odzala.geojson')

cmap = plt.get_cmap("Greens")


fig, axes = plt.subplots(1, 5, figsize=(20, 5))

for i, ax in enumerate(axes):
    with rasterio.open(os.path.join(folder, rasters[i])) as src:
        title = rasters[i].split(".")[0]
        image = src.read(1)
        image = np.ma.masked_where(image == 0, image) 

        extent = src.bounds

        park = park.to_crs(src.crs)

        ax.imshow(image, extent=(extent.left, extent.right, extent.bottom, extent.top), origin='upper', cmap='Greens')
        park.boundary.plot(ax=ax, color='green', linewidth=0.5)
        ax.set_title(title, fontsize=20)
        ax.set_axis_off()

# cbar = fig.colorbar(axes[4].imshow(image, cmap=cmap), ax=axes, orientation="horizontal", fraction=0.046, pad=1, anchor=(0.5, -0.1), ticks=[1, 2, 3, 4, 5])
# cbar.ax.tick_params(labelsize=22)

plt.tight_layout()
# plt.show()
plt.savefig('../data/odzala/plots/species.png', bbox_inches='tight', dpi=300)



# Summarise rasters within polgyon
Here, we attempt to count the number of pixels in each raster.

In [ ]:
from rasterstats import zonal_stats


def summarise_raster(raster_path, year):

    with rasterio.open(raster_path) as src:
        raster = src.read(1)
        affine=src.transform

        # Read the polygon
        park = gpd.read_file('../data/odzala.geojson')

        # Calculate zonal statistics
        stats = zonal_stats(park, raster, affine=affine)[0]

        # Add the year into the dict
        stats['year'] = int(year)

        return stats
    


Create a dataframe of all results by looping through the human-footprint images.

In [ ]:
folder = '../data/odzala/human_footprint_1000/'
rasters = sorted([f for f in os.listdir(folder) if f.endswith(".tif")])

results_list = []

for raster in rasters:
    raster_path = folder+raster
    raster_year = '20' + raster_path.split("_")[4]

    results = summarise_raster(raster_path, raster_year)
    results_list.append(results)

# Create a dataframe
results_df = pd.DataFrame(results_list)

# Save a csv file
results_df.to_csv("../data/odzala/results/human_footprint.csv", index=False)


Then, also for the species rasters

In [ ]:
folder = '../data/odzala/species/Summed/'
rasters = sorted([f for f in os.listdir(folder) if f.endswith(".tif")])

results_list = []

for raster in rasters:
    raster_path = folder+raster
    raster_year = raster.split(".")[0]

    results = summarise_raster(raster_path, raster_year)
    results_list.append(results)

# Create a dataframe
results_df = pd.DataFrame(results_list)

# Save a csv file
results_df.to_csv("../data/odzala/results/species.csv", index=False)



# Export rasters as CSVs

In [2]:
import rasterio
import numpy as np

# Open the raster file using rasterio
with rasterio.open("../data/odzala/human_footprint_1000/odzala_humanfootprint_17_1000.tif") as src:
    # Read the data from the raster into a numpy array
    data = src.read(1)
    # Convert the data to a flattened 1D array
    flat_data = data.flatten()
    # Write the data to a CSV file
    np.savetxt("../data/odzala/results/odzala_humanfootprint_17_1000.csv", data, delimiter=",")